# DATASET CREATION
* Team: 1
* Students: Magnin Jonathan, Nonaca Darja, Shmeis Zeinab, Wang Shu

## LICENSE
### YOLO LICENSE
Version 2, July 29 2016

THIS SOFTWARE LICENSE IS PROVIDED "ALL CAPS" SO THAT YOU KNOW IT IS SUPER
SERIOUS AND YOU DON'T MESS AROUND WITH COPYRIGHT LAW BECAUSE YOU WILL GET IN
TROUBLE HERE ARE SOME OTHER BUZZWORDS COMMONLY IN THESE THINGS WARRANTIES
LIABILITY CONTRACT TORT LIABLE CLAIMS RESTRICTION MERCHANTABILITY. NOW HERE'S
THE REAL LICENSE:

0. Darknet is public domain.
1. Do whatever you want with it.
2. Stop emailing me about it!

## Description
This code uses raw data downloaded from (TODO : put link) to create a proper dataset in CSV files. Since we use both public and private data, only public data are uploaded on this git, which mean that a part of this code cannot work. That is why this code should *NOT BE EXECUTED* and is here only for reading.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
train_filename = "complete_data.csv"
test_filename = "complete_test_data.csv"
generate_test_data = False;

In [ ]:
# example case - DO NOT USE -
header = ["student_name","to_drop1","sct_student","course","prof","to_drop2","course_sct"];
d12_fall_pd = pd.read_csv("data/data2012autumn.csv",names=header)
sct_split = d12_fall_pd["course_sct"].str.split(",",n=1,expand=True)
d12_fall_pd["course_sct"] = sct_split[0]
d12_fall_pd.insert(7,"year",sct_split[1])
d12_fall_pd.head()

In [ ]:
# general case, huge df with every semester concatenated - WHAT YOU WANT -
# years sampled
if generate_test_data :
    years = ["2017", "2018", "2019"];
else :
    years = ["2015","2016"];
header = ["student_name","to_drop1","sct_student","course","prof","to_drop2","course_sct"];

# all other df
all_data = pd.DataFrame(columns=header)
for year in years:
    temp = pd.read_csv("data/data" + year + "autumn.csv",names=header)
    sct_split = temp["course_sct"].str.split(",", n=1, expand=True)
    temp["course_sct"] = sct_split[0]
    temp.insert(7, "year", sct_split[1])
    all_data = pd.concat([all_data,temp], axis=0)
    temp = pd.read_csv("data/data" + year + "spring.csv",names=header)
    sct_split = temp["course_sct"].str.split(",",n=1,expand=True)
    temp["course_sct"] = sct_split[0]
    temp.insert(7,"year",sct_split[1])
    all_data = pd.concat([all_data,temp],axis=0)
    
# drop useless columns
all_data = all_data.drop(["to_drop1", "to_drop2"], axis=1)
    


In [ ]:
all_data['year'].value_counts()

In [ ]:
all_data.iloc[100000] # just to check

In [ ]:
# pip install lxml # in case of error lxml not found 

In [ ]:
#parsing the xml files in data_login to create features.csv
#this block takes time to run, we need to run it only once to generate features.csv file
import os
import csv

pathName = os.getcwd() + '/data_login'
fileNames = os.listdir(pathName)             #put all the files names into a list
#column_list = list(df_login.columns)         #get all the colmn names
col = ['gender', 'student_name', 'nationality'] #select the feature columns

with open('features.csv', 'w', newline='') as csvfile: #open the file where to write
    writer = csv.DictWriter(csvfile, fieldnames=col)                       #initialize the writer
    writer.writeheader()
    for filexls in fileNames:  #iterate through all .xml files 
        df_new = pd.read_html(os.path.join(pathName, filexls), header = 2) # read .xml
        if df_new != []:
            df_new = df_new[0]
            for index, row in df_new.iterrows():  #write the content of a non-empty .xls into an features.csv
                writer.writerow({'gender':str(row['Civilité']) , 'student_name':str(row['Nom Prénom']) , 'nationality':str(row['Nationalité'])})
                
os.system('sed -i -e s/\xa0/\x20/g features.csv') # replace "space" special character by "space" normal character
os.system('sed -i \'\' \'s/nationality /nationality/g\' features.csv')

In [ ]:
features = pd.read_csv('features.csv', sep=',', header=0, encoding='latin1')
#for idx, row in features.iterrows():
#      features.loc[idx, 'student_name'] =features.loc[idx, 'Nom Prénom'].replace('\xa0', ' ')
#data_merged = all_data.merge(features, left_on ='student_name', right_on = 'Nom Prénom',  how = 'inner')
#we need to remove \xa0 from the names in the features before to merge
#also let's do unique the features rows

In [ ]:
features.loc[1, 'student_name'] #debug

In [ ]:
features.drop_duplicates(inplace=True)
features['student_id'] = features.index
features

In [ ]:
complete_data = all_data.merge(features, left_on = "student_name", right_on = "student_name", how = "inner")
complete_data.drop_duplicates(inplace=True)
complete_data

In [ ]:
complete_data[complete_data["student_name"] == "Magnin Jonathan"]

In [ ]:
complete_data.drop(["student_name"], axis=1, inplace=True)
complete_data

In [ ]:
complete_data.to_csv('./complete_test_data.csv', sep = ',')